# Numba CUDA intro

> **GITHUB: https://github.com/daniel-vera-g/numba-cuda-intro**

## Table of contents

- A crash course over the theory: [CUDA components](#Short-summary-of-CUDA-components)
- Use CUDA in with Numba: [CUDA with Numba](#Use-CUDA-with-Numba)

**CUDA concepts in Numba:** 

- What Kernels are and how they work: [Kernels](kernels.ipynb)
- How to manage memory when doing operations: [Memory management](memory-management.ipynb)

## Short summary of CUDA components

1. host: CPU
2. device: GPU
3. host memory: system main memory
4. device memory: onboard memory on a GPU card
5. kernel: a GPU function launched by the host and executed on the device
6. device function: a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)

![](http://upload.wikimedia.org/wikipedia/commons/thumb/5/59/CUDA_processing_flow_%28En%29.PNG/450px-CUDA_processing_flow_%28En%29.PNG)

### CUDA structure

A grid contains blocks with threads:

![](https://www.researchgate.net/profile/Omar-Bouattane/publication/321666991/figure/fig2/AS:572931245260800@1513608861931/Figure-2-Execution-model-of-a-CUDA-program-on-NVidias-GPU-Hierarchy-grid-blocks-and.png)

## Use CUDA with Numba

**NOTE**: If you don't have a CUDA capable GPU, you can activate the _simulation_ mode by starting Jupyterlab with the `NUMBA_ENABLE_CUDASIM=1` ENV:

> `NUMBA_ENABLE_CUDASIM=1 jupyter-lab`

In [1]:
from numba import cuda
print(cuda.gpus) # -> Will probably not work, if you don't have a GPU

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: